In [1]:
from pandas import DataFrame, read_csv
from helpers.preprocess import preprocess_data

import pandas as pd 
import numpy as np

In [2]:
movies_md = r'dataset/movies_tmdbMeta.csv'
movies_md_df = pd.read_csv(movies_md)
pd.set_option('display.max_columns', 50)
movies_md_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords,weighted_rating,cast_size,crew_size,director,revenue_divide_budget
0,False,Toy Story Collection,30000000.0,"['Animation', 'Comedy', 'Family']",http://toystory.disney.com/toy-story,862.0,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,['Pixar Animation Studios'],['United States of America'],1995-10-30,373554033.0,81.0,1,Released,NaN,Toy Story,False,7.7,5415.0,"['Tom Hanks', 'Tim Allen', 'Don Rickles']","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","['jealousy', 'toy', 'boy', 'friendship', 'frie...",7.684645,13,106,John Lasseter,12.451801
1,False,NaN,65000000.0,"['Adventure', 'Fantasy', 'Family']",NaN,8844.0,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"['TriStar Pictures', 'Teitler Film', 'Intersco...",['United States of America'],1995-12-15,262797249.0,104.0,2,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"['Robin Williams', 'Jonathan Hyde', 'Kirsten D...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","['board game', 'disappearance', ""based on chil...",6.876926,26,16,Joe Johnston,4.043035
2,False,Grumpy Old Men Collection,NaN,"['Romance', 'Comedy']",NaN,15602.0,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.712900,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"['Warner Bros.', 'Lancaster Gate']",['United States of America'],1995-12-22,NaN,101.0,1,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"['Walter Matthau', 'Jack Lemmon', 'Ann-Margret']","[{'credit_id': '52fe466a9251416c75077a89', 'de...","['fishing', 'best friend', 'duringcreditssting...",6.162328,7,4,Howard Deutch,NaN
3,False,NaN,16000000.0,"['Comedy', 'Drama', 'Romance']",NaN,31357.0,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,['Twentieth Century Fox Film Corporation'],['United States of America'],1995-12-22,81452156.0,127.0,1,Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"['Whitney Houston', 'Angela Bassett', 'Loretta...","[{'credit_id': '52fe44779251416c91011acb', 'de...","['based on novel', 'interracial relationship',...",5.675590,10,10,Forest Whitaker,5.090760
4,False,Father of the Bride Collection,NaN,['Comedy'],NaN,11862.0,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"['Sandollar Productions', 'Touchstone Pictures']",['United States of America'],1995-02-10,76578911.0,106.0,1,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"['Steve Martin', 'Diane Keaton', 'Martin Short']","[{'credit_id': '52fe44959251416c75039ed7', 'de...","['baby', 'midlife crisis', 'confidence', 'agin...",5.624595,12,7,Charles Shyer,NaN


In [3]:
movies_md_df = preprocess_data(movies_md_df)
movies_md_df.head()

,belongs_to_collection,budget,homepage,overview,popularity,production_countries,runtime,spoken_languages,tagline,title,vote_average,keywords,cast_size,crew_size,production_score,is_english,rating,is_drama,is_comedy,is_thriller,is_action,is_romance,is_adventure,is_crime,is_science_fiction,is_horror,is_family,is_fantasy,is_mystery,is_animation,is_history,is_music,is_war,is_western,is_documentary,is_foreign
0,1,30000000.0,1,"Led by Woody, Andy's toys live happily in his ...",21.946943,[United States of America],81.0,1,NaN,Toy Story,7.7,"['jealousy', 'toy', 'boy', 'friendship', 'frie...",13,106,51.713007,1,3,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0
1,1,65000000.0,1,When siblings Judy and Peter discover an encha...,17.015539,[United States of America],104.0,2,Roll the dice and unleash the excitement!,Jumanji,6.9,"['board game', 'disappearance', ""based on chil...",26,16,47.511209,1,3,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0
2,1,NaN,1,A family wedding reignites the ancient feud be...,11.712900,[United States of America],101.0,1,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,"['fishing', 'best friend', 'duringcreditssting...",7,4,3.483333,1,3,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,16000000.0,1,"Cheated on, mistreated and stepped on, the wom...",3.859495,[United States of America],127.0,1,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,"['based on novel', 'interracial relationship',...",10,10,12.838144,1,2,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,NaN,1,Just when George Banks has recovered from his ...,8.387519,[United States of America],106.0,1,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,"['baby', 'midlife crisis', 'confidence', 'agin...",12,7,24.590805,1,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
y = movies_md_df[['rating']]
y.to_csv('dataset/Y.csv', index=True, index_label='Id')
y.head()

,rating
0,3
1,3
2,3
3,2
4,2


In [5]:
movies_md_df.to_csv('dataset/X.csv', index=True, index_label='Id')

In [6]:
f = movies_md_df[movies_md_df['adult'] == False]
f.shape

KeyError: 'adult'

(5458, 42)